Build a Pass-through composed of three pools of loans, containing a total of 10,000 loans. Each loan, when originated, was exactly $100k in size. 

All the loans in a pool have **identical** characteristics:
- **Pool 1**: 1,000 loans are 15yr fixed-rate 6.40% WAC, 1 months WALA, SMM 0% **(no repayment or curtailment)**
- **Pool 2**: 7,000 loans are 30yr fixed-rate 6.80% WAC, 0 months WALA, SMM 1% **(just prepay entire loan)**
- **Pool 3**: 2,000 loans are 30yr fixed-rate 7.70% WAC, 3 months WALA, SMM 2% **(just curtail)**

- Assume the servicing fee + g-fee is *75bps*.
- For each pool, and each month, calculate the unpaid balance, scheduled principal payments, unscheduled principal or curtailments, interest expense, WAC, WAL, SMM.  
- Calculate the same metrics for the pass-through, and calculate the net coupon each month.
- Explain the drivers of trends in WAC, WAL, SMM

### POOL 1

Since all the loans in pool 1 have identical characteristics, and there are no prepayment or curtailment, then WAC would stay the same and all the loans in pool 1 would have coupon rate being 6.4%.

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

N = 1000
maturity = 15 * 12  # 15 years in months
coupon_rate = 0.064  # 6.40% annual interest
WAC = 0.064
monthly_rate = coupon_rate / 12
initial_balance = 100000
mthly_pymt = monthly_rate * (1 + monthly_rate) ** maturity / ((1 + monthly_rate) ** maturity - 1) * initial_balance
table_p1 = pd.DataFrame({'Months': range(1, maturity + 1)})
begin_balance = [initial_balance]
end_balance = [initial_balance]
interest_pymt = []
scheduled_principal_pymt = []
for i in range(maturity):
    interest_pymt.append(end_balance[-1] * monthly_rate)
    scheduled_principal_pymt.append(mthly_pymt - interest_pymt[-1])
    end_balance.append(begin_balance[-1] - scheduled_principal_pymt[-1])
    begin_balance.append(end_balance[-1])

table_p1['Begin Balance'] = np.array(begin_balance[:-1]) * N
table_p1['Mthly Pymt'] = mthly_pymt * N
table_p1['Interest Pymt'] = np.array(interest_pymt) * N
table_p1['Sche Prin Pymt'] = np.array(scheduled_principal_pymt) * N
table_p1['Unsche Prin Pymt'] = [0] * maturity
table_p1['Total Prin Pymt'] = np.array(scheduled_principal_pymt) + np.array([0] * maturity)
table_p1['End Balance'] = np.array(end_balance[1:]) * N
table_p1['WAC'] = [WAC] * maturity
table_p1['SMM'] = [0] * maturity
WAL = []
for i in range(maturity):
    WAL.append(np.sum(np.array(scheduled_principal_pymt)[i:] * np.array(range(1, maturity - i + 1))) / end_balance[i] / 12)
table_p1['WAL'] = WAL
table_p1.set_index('Months',inplace=True)
table_p1

,Begin Balance,Mthly Pymt,Interest Pymt,Sche Prin Pymt,Unsche Prin Pymt,Total Prin Pymt,End Balance,WAC,SMM,WAL
Months,,,,,,,,,,
1,1.000000e+08,865619.405163,533333.333333,332286.071830,0,332.286072,9.966771e+07,0.064,0,8.720546
2,9.966771e+07,865619.405163,531561.140950,334058.264213,0,334.058264,9.933366e+07,0.064,0,8.666008
3,9.933366e+07,865619.405163,529779.496874,335839.908289,0,335.839908,9.899782e+07,0.064,0,8.611539
4,9.899782e+07,865619.405163,527988.350697,337631.054467,0,337.631054,9.866018e+07,0.064,0,8.557136
5,9.866018e+07,865619.405163,526187.651740,339431.753424,0,339.431753,9.832075e+07,0.064,0,8.502802
6,9.832075e+07,865619.405163,524377.349055,341242.056109,0,341.242056,9.797951e+07,0.064,0,8.448535
7,9.797951e+07,865619.405163,522557.391422,343062.013741,0,343.062014,9.763645e+07,0.064,0,8.394336
8,9.763645e+07,865619.405163,520727.727349,344891.677815,0,344.891678,9.729156e+07,0.064,0,8.340204
9,9.729156e+07,865619.405163,518888.305067,346731.100096,0,346.731100,9.694483e+07,0.064,0,8.286141


### POOL 2 

Pool 2 borrowers just prepay entire loan (nobody curtails). Each month, round down the number of borrowers to a whole number, e.g. in the first month, 70 prepay, leaving 6930 borrowers. The next month 69 borrowers prepay, not 69.3!

Note that the WAC of pool 2 (i.e. *6.80%*) will remain unchanged since every loan in the pool has the identical characteristics, we have 
$$
WAC_k = \frac{\sum_{i=1}^L(Coupon_{l,k}*balance_{l,k})}{\sum_{i=1}^L balance_{l,k}} = \frac{Coupon\sum_{i=1}^L balance_{l,k})}{\sum_{i=1}^L balance_{l,k}} = Coupon.
$$
Actually here we are assuming SMM stays the same throughout the maturity.

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

N = 7000
maturity = 30 * 12
coupon_rate = 0.068
WAC = 0.068
SMM = 0.01
monthly_rate = coupon_rate / 12
initial_balance = 100000 * N
mthly_pymt_indiv = monthly_rate * (1 + monthly_rate) ** maturity / ((1 + monthly_rate) ** maturity - 1) * 100000
mthly_pymt = []
table_p2 = pd.DataFrame({'Months': range(1, maturity + 1)})
begin_balance = [initial_balance]
end_balance = [initial_balance]
interest_pymt = []
scheduled_principal_pymt = []
unscheduled_principal_pymt = []
SMM_his = []
left = N
for i in range(maturity):
    interest_pymt.append(end_balance[-1] * monthly_rate)
    num_prepay = int(left * SMM)
    mthly_pymt.append(mthly_pymt_indiv * (left-num_prepay) + num_prepay*end_balance[-1]/left)
    scheduled_principal_pymt.append(mthly_pymt_indiv*left - interest_pymt[-1])
    unscheduled_principal_pymt.append(num_prepay / left * (end_balance[-1]-scheduled_principal_pymt[-1]))
    end_balance.append(begin_balance[-1] - scheduled_principal_pymt[-1] - unscheduled_principal_pymt[-1])
    begin_balance.append(end_balance[-1])
    SMM_his.append(round(unscheduled_principal_pymt[-1] / end_balance[-1] , 4))
    left -= num_prepay


table_p2['Begin Balance'] = begin_balance[:-1]
table_p2['Mthly Pymt'] = mthly_pymt
table_p2['Interest Payment'] = interest_pymt
table_p2['Sche Prin Pymt'] = scheduled_principal_pymt
table_p2['Unsche Prin Pymt'] = unscheduled_principal_pymt
table_p2['Total Prin Pymt'] = np.array(scheduled_principal_pymt) + np.array(unscheduled_principal_pymt)
table_p2['End Balance'] = end_balance[1:]
table_p2['WAC'] = [WAC] * maturity
table_p2['SMM'] = SMM_his
table_p2['Cum Interest'] = np.cumsum(interest_pymt)
WAL = []
for i in range(maturity):
    WAL.append(
        np.sum(np.array(table_p2['Total Prin Pymt'])[i:] * np.array(range(1, maturity - i + 1))) / end_balance[i] / 12)
table_p2['WAL'] = WAL
table_p2.set_index('Months',inplace=True)
table_p2

,Begin Balance,Mthly Pymt,Interest Payment,Sche Prin Pymt,Unsche Prin Pymt,Total Prin Pymt,End Balance,WAC,SMM,Cum Interest,WAL
Months,,,,,,,,,,,
1,7.000000e+08,1.151784e+07,3.966667e+06,596809.650488,6.994032e+06,7.590842e+06,6.924092e+08,0.068,0.0101,3.966667e+06,7.152844
2,6.924092e+08,1.136698e+07,3.923652e+06,594189.656122,6.888201e+06,7.482391e+06,6.849268e+08,0.068,0.0101,7.890319e+06,7.147013
3,6.849268e+08,1.121690e+07,3.881252e+06,591607.031788,6.782509e+06,7.374116e+06,6.775527e+08,0.068,0.0100,1.177157e+07,7.140846
4,6.775527e+08,1.106761e+07,3.839465e+06,589062.773767,6.676956e+06,7.266019e+06,6.702866e+08,0.068,0.0100,1.561104e+07,7.134323
5,6.702866e+08,1.101813e+07,3.798291e+06,586557.891200,6.671113e+06,7.257671e+06,6.630290e+08,0.068,0.0101,1.940933e+07,7.127423
6,6.630290e+08,1.086969e+07,3.757164e+06,584005.704503,6.565756e+06,7.149761e+06,6.558792e+08,0.068,0.0100,2.316649e+07,7.121196
7,6.558792e+08,1.072204e+07,3.716649e+06,581493.956686,6.460542e+06,7.042036e+06,6.488372e+08,0.068,0.0100,2.688314e+07,7.114583
8,6.488372e+08,1.067393e+07,3.676744e+06,579023.687804,6.454776e+06,7.033800e+06,6.418034e+08,0.068,0.0101,3.055988e+07,7.107562
9,6.418034e+08,1.052714e+07,3.636886e+06,576506.750125,6.349763e+06,6.926270e+06,6.348771e+08,0.068,0.0100,3.419677e+07,7.101210


### POOL 3

Pool 3 borrowers just curtail (nobody prepays the entire loan, except for the very last payment).

Note that even under the prepayment circumstance, the WAC of pool 3(i.e. *7.70%*) will still remain unchanged, as explained above.

In [3]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)

N = 2000
maturity = 30 * 12
coupon_rate = 0.077
WAC = 0.077
SMM = 0.02
monthly_rate = coupon_rate / 12
initial_balance = 100000 * N
mthly_pymt = monthly_rate * (1 + monthly_rate) ** maturity / ((1 + monthly_rate) ** maturity - 1) * initial_balance
table_p3 = pd.DataFrame()
begin_balance = [initial_balance]
end_balance = [initial_balance]
interest_pymt = []
scheduled_principal_pymt = []
unscheduled_principal_pymt = []
while end_balance[-1] > 0:
    interest_pymt.append(end_balance[-1] * monthly_rate)
    scheduled_principal_pymt.append(mthly_pymt - interest_pymt[-1])
    unscheduled_principal_pymt.append(SMM * end_balance[-1])
    temp = begin_balance[-1] - scheduled_principal_pymt[-1] - unscheduled_principal_pymt[-1]
    end_balance.append(temp if temp > 0 else 0)
    begin_balance.append(end_balance[-1])

table_p3['Months'] = list(range(1,len(interest_pymt)+1))
table_p3['Begin Balance'] = begin_balance[:-1]
table_p3['Mthly Pymt'] = [mthly_pymt] * len(interest_pymt)
table_p3['Interest Payment'] = interest_pymt
table_p3['Sche Prin Pymt'] = scheduled_principal_pymt
table_p3['Unsche Prin Pymt'] = unscheduled_principal_pymt
table_p3['Total Prin Pymt'] = np.array(scheduled_principal_pymt) + np.array(unscheduled_principal_pymt)
table_p3['End Balance'] = end_balance[1:]
table_p3['WAC'] = [WAC] * len(interest_pymt)
table_p3['SMM'] = [SMM] * len(interest_pymt)
WAL = []
for i in range(len(interest_pymt)):
    WAL.append(
        np.sum(np.array(table_p3['Total Prin Pymt'])[i:] * np.array(range(1, len(interest_pymt) - i + 1))) / end_balance[i] / 12)
table_p3['WAL'] = WAL
table_p3.set_index('Months', inplace=True)
table_p3


,Begin Balance,Mthly Pymt,Interest Payment,Sche Prin Pymt,Unsche Prin Pymt,Total Prin Pymt,End Balance,WAC,SMM,WAL
Months,,,,,,,,,,
1,2.000000e+08,1.425920e+06,1.283333e+06,1.425870e+05,4.000000e+06,4.142587e+06,1.958574e+08,0.077,0.02,2.724973
2,1.958574e+08,1.425920e+06,1.256752e+06,1.691686e+05,3.917148e+06,4.086317e+06,1.917711e+08,0.077,0.02,2.697501
3,1.917711e+08,1.425920e+06,1.230531e+06,1.953891e+05,3.835422e+06,4.030811e+06,1.877403e+08,0.077,0.02,2.669860
4,1.877403e+08,1.425920e+06,1.204667e+06,2.212535e+05,3.754806e+06,3.976059e+06,1.837642e+08,0.077,0.02,2.642047
5,1.837642e+08,1.425920e+06,1.179154e+06,2.467666e+05,3.675285e+06,3.922051e+06,1.798422e+08,0.077,0.02,2.614064
6,1.798422e+08,1.425920e+06,1.153987e+06,2.719331e+05,3.596843e+06,3.868777e+06,1.759734e+08,0.077,0.02,2.585909
7,1.759734e+08,1.425920e+06,1.129163e+06,2.967577e+05,3.519468e+06,3.816226e+06,1.721572e+08,0.077,0.02,2.557582
8,1.721572e+08,1.425920e+06,1.104675e+06,3.212452e+05,3.443143e+06,3.764389e+06,1.683928e+08,0.077,0.02,2.529082
9,1.683928e+08,1.425920e+06,1.080520e+06,3.454000e+05,3.367856e+06,3.713256e+06,1.646795e+08,0.077,0.02,2.500410


## Pass-Through

In [4]:
# pool 1 WALA = 1M, pool 2 WALA = 0M, pool 3 WALA = 3M
pool1_issued = table_p1.iloc[1:,range(7)]
pool2_issued = table_p2.iloc[:,range(7)]
pool3_issued = table_p3.iloc[3:,range(7)]
# Make sure the indexes are aligned to proceed
pool1_issued.index = pd.RangeIndex(start=0, stop=len(pool1_issued), step=1)
pool2_issued.index = pd.RangeIndex(start=0, stop=len(pool2_issued), step=1)
pool3_issued.index = pd.RangeIndex(start=0, stop=len(pool3_issued), step=1)
# Align 3 pools to the longest pool loan age
pool1_issued_aligned = pool1_issued.reindex_like(pool2_issued).fillna(0)
pool2_issued_aligned = pool2_issued.reindex_like(pool2_issued).fillna(0)
pool3_issued_aligned = pool3_issued.reindex_like(pool2_issued).fillna(0)
pass_through = pool1_issued_aligned+pool2_issued_aligned+pool3_issued_aligned
pass_through

,Begin Balance,Mthly Pymt,Interest Payment,Sche Prin Pymt,Unsche Prin Pymt,Total Prin Pymt,End Balance
0,9.874080e+08,1.380938e+07,5.171333e+06,1.152121e+06,1.074884e+07,1.156723e+07,9.755070e+08
1,9.755070e+08,1.365852e+07,5.102806e+06,1.176796e+06,1.056349e+07,1.140478e+07,9.637668e+08
2,9.637668e+08,1.350844e+07,5.035239e+06,1.201171e+06,1.037935e+07,1.124323e+07,9.521862e+08
3,9.521862e+08,1.335915e+07,4.968628e+06,1.225252e+06,1.019642e+07,1.108258e+07,9.407646e+08
4,9.407646e+08,1.330967e+07,4.902966e+06,1.249045e+06,1.011426e+07,1.102240e+07,9.294013e+08
5,9.294013e+08,1.316123e+07,4.837684e+06,1.272468e+06,9.933611e+06,1.086336e+07,9.181952e+08
6,9.181952e+08,1.301358e+07,4.773342e+06,1.295612e+06,9.754132e+06,1.070520e+07,9.071454e+08
7,9.071454e+08,1.296547e+07,4.709935e+06,1.318485e+06,9.675110e+06,1.064721e+07,8.961518e+08
8,8.961518e+08,1.281868e+07,4.646892e+06,1.341001e+06,9.497836e+06,1.049060e+07,8.853130e+08
9,8.853130e+08,1.267269e+07,4.584775e+06,1.363254e+06,9.321690e+06,1.033486e+07,8.746281e+08


According to the proportion of each pool, the SMM of the pass through is:

$$
SMM = \frac{0*1000+0.01*7000+0.02*2000}{1000+7000+2000}=0.011
$$

In the last 3 months, the pool 1,3 has ended:

$$
SMM = 0.01
$$

In [5]:
WAL = []
for i in range(360):
    WAL.append(
        np.sum(np.array(pass_through['Total Prin Pymt'])[i:] * np.array(range(1, 360 - i + 1))) / pass_through.iloc[i,6] / 12)
WAL[-1] = 0
pass_through['WAL'] = WAL
pass_through['SMM'] = [0.011] * 357 + [0.01]*3

zeros_df = pd.DataFrame(0., index=range(181,361), columns=table_p1.columns)
table_p1 = pd.concat([table_p1, zeros_df])
zeros_df_2 = pd.DataFrame(0., index=range(79,361), columns=table_p3.columns)
table_p3 = pd.concat([table_p3, zeros_df_2])
pass_through['WAC'] = list((table_p1['WAC'] * table_p1['End Balance'] + table_p2['WAC'] * table_p2['End Balance']
                            + table_p3['WAC'] * table_p3['End Balance'])/(table_p1['End Balance']+table_p2['End Balance']+table_p3['End Balance']))

In [6]:
pass_through

,Begin Balance,Mthly Pymt,Interest Payment,Sche Prin Pymt,Unsche Prin Pymt,Total Prin Pymt,End Balance,WAL,SMM,WAC
0,9.874080e+08,1.380938e+07,5.171333e+06,1.152121e+06,1.074884e+07,1.156723e+07,9.755070e+08,5.642064,0.011,0.069381
1,9.755070e+08,1.365852e+07,5.102806e+06,1.176796e+06,1.056349e+07,1.140478e+07,9.637668e+08,5.634023,0.011,0.069361
2,9.637668e+08,1.350844e+07,5.035239e+06,1.201171e+06,1.037935e+07,1.124323e+07,9.521862e+08,5.625852,0.011,0.069342
3,9.521862e+08,1.335915e+07,4.968628e+06,1.225252e+06,1.019642e+07,1.108258e+07,9.407646e+08,5.617542,0.011,0.069322
4,9.407646e+08,1.330967e+07,4.902966e+06,1.249045e+06,1.011426e+07,1.102240e+07,9.294013e+08,5.609684,0.011,0.069302
5,9.294013e+08,1.316123e+07,4.837684e+06,1.272468e+06,9.933611e+06,1.086336e+07,9.181952e+08,5.601677,0.011,0.069282
6,9.181952e+08,1.301358e+07,4.773342e+06,1.295612e+06,9.754132e+06,1.070520e+07,9.071454e+08,5.593521,0.011,0.069262
7,9.071454e+08,1.296547e+07,4.709935e+06,1.318485e+06,9.675110e+06,1.064721e+07,8.961518e+08,5.585825,0.011,0.069241
8,8.961518e+08,1.281868e+07,4.646892e+06,1.341001e+06,9.497836e+06,1.049060e+07,8.853130e+08,5.577969,0.011,0.069221
9,8.853130e+08,1.267269e+07,4.584775e+06,1.363254e+06,9.321690e+06,1.033486e+07,8.746281e+08,5.569954,0.011,0.069200


In [7]:
fee = 0.0075
pass_through['Net Mthly Coupon %'] = ((pass_through['Interest Payment'] / pass_through['Begin Balance'])*12 - fee) * 100
pass_through

,Begin Balance,Mthly Pymt,Interest Payment,Sche Prin Pymt,Unsche Prin Pymt,Total Prin Pymt,End Balance,WAL,SMM,WAC,Net Mthly Coupon %
0,9.874080e+08,1.380938e+07,5.171333e+06,1.152121e+06,1.074884e+07,1.156723e+07,9.755070e+08,5.642064,0.011,0.069381,5.534738
1,9.755070e+08,1.365852e+07,5.102806e+06,1.176796e+06,1.056349e+07,1.140478e+07,9.637668e+08,5.634023,0.011,0.069361,5.527112
2,9.637668e+08,1.350844e+07,5.035239e+06,1.201171e+06,1.037935e+07,1.124323e+07,9.521862e+08,5.625852,0.011,0.069342,5.519449
3,9.521862e+08,1.335915e+07,4.968628e+06,1.225252e+06,1.019642e+07,1.108258e+07,9.407646e+08,5.617542,0.011,0.069322,5.511751
4,9.407646e+08,1.330967e+07,4.902966e+06,1.249045e+06,1.011426e+07,1.102240e+07,9.294013e+08,5.609684,0.011,0.069302,5.504019
5,9.294013e+08,1.316123e+07,4.837684e+06,1.272468e+06,9.933611e+06,1.086336e+07,9.181952e+08,5.601677,0.011,0.069282,5.496195
6,9.181952e+08,1.301358e+07,4.773342e+06,1.295612e+06,9.754132e+06,1.070520e+07,9.071454e+08,5.593521,0.011,0.069262,5.488337
7,9.071454e+08,1.296547e+07,4.709935e+06,1.318485e+06,9.675110e+06,1.064721e+07,8.961518e+08,5.585825,0.011,0.069241,5.480447
8,8.961518e+08,1.281868e+07,4.646892e+06,1.341001e+06,9.497836e+06,1.049060e+07,8.853130e+08,5.577969,0.011,0.069221,5.472462
9,8.853130e+08,1.267269e+07,4.584775e+06,1.363254e+06,9.321690e+06,1.033486e+07,8.746281e+08,5.569954,0.011,0.069200,5.464446


#### The drivers of trends in WAC, WAL, SMM

- **WAC**: Prepayments and curtailments in the pool can affect WAC due to the proportion of loans with various coupon rates under the fixed-rate scenario. If the prepayment reduces the relative proportion with higher coupon rates, the WAC of the entire pass-through decreases; if the relative proportion with lower coupon rates decreases, the WAC of the entire pass-through increases.

- **WAL**: WAL is substantially affected by the speed of prepayments and curtailments. An rise in prepayments will reduce WAL because principal is repayed sooner than expected.  It is also influenced by the loan terms and the loan age (WALA). For example, a pool of 30-year loans will have a longer WAL than a pool of 15-year loans, and the age of the loans in the pool (WALA) at the time of pass-through issuance will influence the WAL. 

- **SMM**: Since each pool assumed a fixed-rate SMM, the only factors influencing SMM in this instance are the loan term and WALA. However, certain factors such as interest rate movements do have an impact on SMM. As borrowers want to take advantage of reduced rates, dropping interest rates may result in increased refinancing rates, which would increase SMM.